In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### P.V.C <br> x*y''' + 3*y' + x²*y = e**x<br>y(0) = 1; y(1) = -1

In [70]:
def f_expr(expression):
    #Função para criar uma função a partir de uma expressão
    ## Entrada: expressão em formato de string
    ## Saída: função em formato de lambda
    
    return eval(f'lambda x, y: {expression}')   

def thomas_algorithm(A, b):
    """
    Implementação do algoritmo de Thomas para resolver um sistema linear tridiagonal.
    
    Parâmetros:
        - A: Matriz tridiagonal
        - b: Vetor do lado direito do sistema
        
    Retorna:
        - x: Solução do sistema
    """
    n = len(b)
    c, d = np.diag(A, k=-1).copy(), np.diag(A).copy()
    
    # Eliminação direta
    for i in range(1, n):
        m = c[i-1] / d[i-1]
        d[i] -= m * A[i-1, i]
        b[i] -= m * b[i-1]
    
    # Substituição inversa
    x = np.zeros_like(b)
    x[-1] = b[-1] / d[-1]
    for i in range(n - 2, -1, -1):
        x[i] = (b[i] - c[i] * x[i+1]) / d[i]
    
    return x


def solve_pvc(f, alpha, beta, a, b, N):
    """
    Resolve um problema de valor de contorno com condições de Dirichlet
    usando o método de diferenças finitas e o algoritmo de Thomas.
    
    Parâmetros:
        - f: Função que define o lado direito da equação diferencial f(x)
        - alpha: Condição de contorno Dirichlet y(a)
        - beta: Condição de contorno Dirichlet y(b)
        - a: Ponto inicial do intervalo
        - b: Ponto final do intervalo
        - N: Número de pontos de grade
        
    Retorna:
        - x: Array com os pontos de grade
        - u: Array com as soluções nos pontos de grade
    """
    h = (b - a) / (N + 1)
    
    # Pontos de grade
    x = np.linspace(a, b, N + 2)
    
    # Matriz tridiagonal
    A = np.zeros((N, N))
    rhs = np.zeros(N)
    
    for i in range(1, N + 1):
        x_i = a + i * h  # Calcular o valor de x no ponto de grade i
        A[i - 1, i - 1] = -2 / h**2 + x_i**2
        # Calcular f(x_i, [y_i-1, y_i, y_i+1])
        rhs[i - 1] = f(x_i, [0, 0, 0]) * h**2  
        if i > 1:
            A[i - 1, i - 2] = 1 / h**2 - 1 / (2 * h * x_i)
    
    # Condições de contorno
    rhs[0] -= alpha / h**2
    rhs[-1] -= beta / h**2
    
    # Resolvendo o sistema linear tridiagonal com o algoritmo de Thomas
    u = thomas_algorithm(A, rhs)
    
    return x, u



# Definição da função f(x)
def f_expr(expression):
    # Função para criar uma função a partir de uma expressão
    # Entrada: expressão em formato de string
    # Saída: função em formato de lambda
    return eval(f'lambda x, y: {expression}')

# Expressão da equação diferencial
f = f_expr("x * (y[2] - 2 * y[1] + y[0]) / h**2 + 3 * (y[2] - y[0]) / (2 * h) + x**2 * y[1] - np.exp(x)")

# Condições de contorno
alpha = 1  # y(0) = 1
beta = -1  # y(1) = -1
a = 0
b = 1
N = 5  # Número de pontos de grade
h = (b - a) / (N + 1)  # Espaçamento de grade

# Resolvendo o PVC
x, u = solve_pvc(f, alpha, beta, a, b, N)

# Imprimindo os resultados de forma formatada
print("Pontos de grade (x):")
print(x)

print("\nSoluções nos pontos de grade (u):")
for i in range(len(u)):
    print(f"x = {x[i+1]:.3f}, u = {u[i]:.6f}")


Pontos de grade (x):
[0.         0.16666667 0.33333333 0.5        0.66666667 0.83333333
 1.        ]

Soluções nos pontos de grade (u):
x = 0.167, u = 0.571113
x = 0.333, u = 0.187831
x = 0.500, u = -0.001777
x = 0.667, u = -0.185096
x = 0.833, u = -0.487731


In [80]:
import numpy as np

def f_expr(expression):
    """
    Função para criar uma função a partir de uma expressão.
    
    Parâmetros:
        - expression: Expressão em formato de string
    
    Retorna:
        - Função em formato de lambda
    """
    return eval(f'lambda x, y, y_x, y_xx: {expression}')

def thomas_algorithm(A, b):
    """
    Implementação do algoritmo de Thomas para resolver um sistema linear tridiagonal.
    
    Parâmetros:
        - A: Matriz tridiagonal
        - b: Vetor do lado direito do sistema
        
    Retorna:
        - x: Solução do sistema
    """
    n = len(b)
    c, d = np.zeros(n - 1), np.zeros(n)
    
    # Decifrando os termos da matriz tridiagonal
    c[:] = A.diagonal(offset=-1)
    d[:] = A.diagonal()
    
    # Eliminação direta
    for i in range(1, n):
        m = c[i-1] / d[i-1]
        d[i] -= m * c[i-1]
        b[i] -= m * b[i-1]
    
    # Substituição inversa
    x = np.zeros_like(b)
    x[-1] = b[-1] / d[-1]
    for i in range(n - 2, -1, -1):
        x[i] = (b[i] - c[i] * x[i+1]) / d[i]
    
    return x

def solve_robin_bvp(f, alpha, beta, gamma, delta, a, b, N):
    """
    Resolve um problema de valor de contorno com condições de Robin
    usando o método de diferenças finitas e o algoritmo de Thomas.
    
    Parâmetros:
        - f: Função que define a equação diferencial f(x, y, y_x, y_xx)
        - alpha: Coeficiente da condição de contorno Robin y'(a) + alpha * y(a) = gamma
        - beta: Coeficiente da condição de contorno Robin y'(b) + beta * y(b) = delta
        - gamma: Termo independente da condição de contorno Robin y'(a) + alpha * y(a) = gamma
        - delta: Termo independente da condição de contorno Robin y'(b) + beta * y(b) = delta
        - a: Ponto inicial do intervalo
        - b: Ponto final do intervalo
        - N: Número de pontos de grade
        
    Retorna:
        - x: Array com os pontos de grade
        - u: Array com as soluções nos pontos de grade
    """
    h = (b - a) / (N + 1)
    
    # Pontos de grade
    x = np.linspace(a, b, N + 2)
    
    # Inicialização de u
    u = np.zeros(N + 2)
    
    # Termos do lado direito da equação
    rhs = np.zeros(N)  # Ajuste do tamanho do vetor rhs
    
    for i in range(1, N + 1):
        rhs[i - 1] = h**2 * f(x[i], u[i], (u[i+1] - u[i-1]) / (2 * h), (u[i+1] - 2 * u[i] + u[i-1]) / h**2)
    
    # Matriz tridiagonal
    A = np.diag(-2 * np.ones(N)) + np.diag(np.ones(N - 1), k=-1) + np.diag(np.ones(N - 1), k=1)  # Ajuste do tamanho da matriz A
    
    # Condições de contorno
    A[0, 0] += alpha * h
    A[0, 1] -= alpha
    A[-1, -1] += beta * h
    A[-1, -2] -= beta
    
    rhs[0] -= gamma * alpha * h
    rhs[-1] -= delta * beta * h
    
    # Resolvendo o sistema linear tridiagonal com o algoritmo de Thomas
    u[1:-1] = thomas_algorithm(A, rhs)
    
    return x, u


# Expressão da equação diferencial
f = f_expr("x * y_xx + 3 * y_x + x**2 * y - np.exp(x)")

# Condições de contorno
alpha = 1
beta = 0
gamma = 1
delta = -1

# Intervalo e número de pontos de grade
a = 0
b = 1
N = 5

# Resolvendo o PVC com condições de contorno de Robin
x, u = solve_robin_bvp(f, alpha, beta, gamma, delta, a, b, N)

# Imprimindo os resultados de forma formatada
print("Pontos de grade (x):")
print(x)

print("\nSoluções nos pontos de grade (u):")
for i in range(len(u)):
    print(f"x = {x[i]:.3f}, u = {u[i]:.6f}")



Pontos de grade (x):
[0.         0.16666667 0.33333333 0.5        0.66666667 0.83333333
 1.        ]

Soluções nos pontos de grade (u):
x = 0.000, u = 0.000000
x = 0.167, u = 0.282967
x = 0.333, u = 0.319291
x = 0.500, u = 0.316847
x = 0.667, u = 0.268606
x = 0.833, u = 0.166261
x = 1.000, u = 0.000000


In [81]:
import numpy as np

def f_expr(expression):
    """
    Função para criar uma função a partir de uma expressão.
    
    Parâmetros:
        - expression: Expressão em formato de string
    
    Retorna:
        - Função em formato de lambda
    """
    return eval(f'lambda x, y, y_x, y_xx: {expression}')

def thomas_algorithm(A, b):
    """
    Implementação do algoritmo de Thomas para resolver um sistema linear tridiagonal.
    
    Parâmetros:
        - A: Matriz tridiagonal
        - b: Vetor do lado direito do sistema
        
    Retorna:
        - x: Solução do sistema
    """
    n = len(b)
    c, d = np.zeros(n - 1), np.zeros(n)
    
    # Decifrando os termos da matriz tridiagonal
    c[:] = A.diagonal(offset=-1)
    d[:] = A.diagonal()
    
    # Eliminação direta
    for i in range(1, n):
        m = c[i-1] / d[i-1]
        d[i] -= m * c[i-1]
        b[i] -= m * b[i-1]
    
    # Substituição inversa
    x = np.zeros_like(b)
    x[-1] = b[-1] / d[-1]
    for i in range(n - 2, -1, -1):
        x[i] = (b[i] - c[i] * x[i+1]) / d[i]
    
    return x

def solve_robin_bvp_newman(f, alpha, beta, gamma, delta, a, b, N):
    """
    Resolve um problema de valor de contorno com condições de Robin
    usando o método de diferenças finitas de Newman.
    
    Parâmetros:
        - f: Função que define a equação diferencial f(x, y, y_x, y_xx)
        - alpha: Coeficiente da condição de contorno Robin y'(a) + alpha * y(a) = gamma
        - beta: Coeficiente da condição de contorno Robin y'(b) + beta * y(b) = delta
        - gamma: Termo independente da condição de contorno Robin y'(a) + alpha * y(a) = gamma
        - delta: Termo independente da condição de contorno Robin y'(b) + beta * y(b) = delta
        - a: Ponto inicial do intervalo
        - b: Ponto final do intervalo
        - N: Número de pontos de grade
        
    Retorna:
        - x: Array com os pontos de grade
        - u: Array com as soluções nos pontos de grade
    """
    h = (b - a) / (N + 1)
    
    # Pontos de grade
    x = np.linspace(a, b, N + 2)
    
    # Inicialização de u
    u = np.zeros(N + 2)
    
    # Termos do lado direito da equação
    rhs = np.array([h**2 * f(xi, u[i], (u[i+1] - u[i-1]) / (2 * h), (u[i+1] - 2 * u[i] + u[i-1]) / h**2) for i, xi in enumerate(x[1:-1])])
    
    # Matriz tridiagonal
    A = np.diag(np.ones(N - 1), k=-1) - 2 * np.diag(np.ones(N)) + np.diag(np.ones(N - 1), k=1)
    
    # Condições de contorno
    rhs[0] -= alpha * gamma
    rhs[-1] -= beta * delta
    
    # Modificando a matriz tridiagonal para levar em conta as condições de contorno de Robin
    A[0, 0] -= alpha / h
    A[0, 1] += alpha / h
    A[-1, -1] += beta / h
    A[-1, -2] -= beta / h
    
    # Resolvendo o sistema linear tridiagonal com o algoritmo de Thomas
    u[1:-1] = thomas_algorithm(A, rhs)
    
    return x, u


# Expressão da equação diferencial
f = f_expr("x * y_xx + 3 * y_x + x**2 * y - np.exp(x)")

# Condições de contorno
alpha = 1
beta = 0
gamma = 1
delta = -1

# Intervalo e número de pontos de grade
a = 0
b = 1
N = 5

# Resolvendo o PVC com condições de contorno de Robin
x, u = solve_robin_bvp(f, alpha, beta, gamma, delta, a, b, N)

# Imprimindo os resultados de forma formatada
print("Pontos de grade (x):")
print(x)

print("\nSoluções nos pontos de grade (u):")
for i in range(len(u)):
    print(f"x = {x[i]:.3f}, u = {u[i]:.6f}")



Pontos de grade (x):
[0.         0.16666667 0.33333333 0.5        0.66666667 0.83333333
 1.        ]

Soluções nos pontos de grade (u):
x = 0.000, u = 0.000000
x = 0.167, u = 0.282967
x = 0.333, u = 0.319291
x = 0.500, u = 0.316847
x = 0.667, u = 0.268606
x = 0.833, u = 0.166261
x = 1.000, u = 0.000000
